# OAC-DE tutorial

In this tutorial, we'll run the OAC-DE on a testing function from *bbob2015* benchmark [1]. The definition of all testing functions from this benchmark is available [here.](http://numbbo.github.io/coco/testsuites/bbob)

## Step 1

Let's start by selecting a testing function from *bbob2015* benchmark, you'll get an object of `<class Benchmark>`:

In [1]:
import oacde
import time
import pandas as pd
pd.set_option('display.max_columns', 4)
pd.set_option('display.width', 1000)


tik = time.time()
seed = 0 # The random seed is fixed ONLY for testing


benchmark = "bbob2015"
D = 10
funID = 7
problem = oacde.problem.Benchmark(benchmark_set=benchmark, D=D, funID=funID, instanceID=1)
print(problem)

## Step 2

Next, we need to initialize the DE's hyperparameters:

In [2]:
NP = 24
config = oacde.publics.parameterize.DE(seed = seed,
                                       N = NP,
                                       mutation='de/rand/1'
                                       )
print(config)

`_rng`: RandomState(MT19937)
`seed`: 0
`_N`: Number of individuals
	Type: Integer
	Value: 24
	Range: [0, 9223372036854775808)
`_initial_scheme`: Initialization scheme
	Type: String
	Value: latin_hypercube
	Allowable options: ('latin_hypercube', 'random')
	Default: latin_hypercube
`_update_scheme`: Update scheme
	Type: String
	Value: immediate
	Allowable options: ('immediate', 'deferred')
	Default: immediate
`_record_position`: Position recording
	Type: Boolean
	Value: False
	Allowable options: (True, False)
	Default: False
`_F`: Scaling factor
	Type: Float
	Value: 0.75
	Range: [0.0, 2.0)
	Default: 0.75
`_CR`: Crossover rate
	Type: Float
	Value: 0.7
	Range: [0.0, 1.0)
	Default: 0.7
`_mutation`: Mutation scheme
	Type: String
	Value: de/rand/1
	Allowable options: ('de/rand/1', 'de/rand/2', 'de/best/1', 'de/best/2', 'de/current/1', 'de/rand-to-best/1', 'de/current-to-best/1', 'de/current-to-rand/1', 'de/current-to-best/2', 'de/current-to-pbest/1', 'de/rand-to-pbest/1')
	Default: de/best/1


As well as stop conditions:

In [3]:
stop = oacde.publics.parameterize.StopCondition(max_FES=1000.*NP, max_iter=None, delta_ftarget=1e-8)
print(stop)

`_max_FES`: Maximum Function Evaluations
	Type: Float
	Value: 24000.0
	Range: [1.0, inf)
	Default: 1e+308
`_max_iter`: Maximum Iterations
	Type: Float
	Value: 1e+308
	Range: [1.0, inf)
	Default: 1e+308
`_delta_ftarget`: Difference between the target and the theoretical optimum
	Type: Float
	Value: 1e-08
	Range: [0.0, inf)
	Default: 2.2250738585072014e-308
`_ftarget`: Target value of the objective function
	Type: Float
	Value: None
	Range: [-inf, inf)
`_condition_limit`: Target value of the objective function
	Type: Float
	Value: 10000000.0
	Range: [0.0, inf)
	Default: 10000000.0


## Step 3

Now, you can personalize the OAC-DE's parameters such as the mutation operators, the crossover operators, ...

In [4]:
optimizer = oacde.OACDE(opt_problem=problem,
                        algo_parameters=config,
                        stop_conditions=stop,
                        mut_pool=[
                        "de/rand/1",
                        "de/rand/2",
                        "de/best/2",
                        "de/current-to-best/2",
                        "de/current-to-rand/1",
                        "de/current-to-pbest/1",
                        ],
                        cx_pool=["bin", "exp"],
                        tuning_level="individual_level",
                        F_range=(0.0, 1.0),
                        CR_range=(0.0, 1.0),
                        model_utilization_strategy="success",
                        stagnation_gen=10,
                        KDE_kernel="gaussian",
                        KDE_width=1/3,
                        KDE_max_size=None,
                        MAB_algo="ucb",
                        MAB_value_method="sample_average",
                        epsilon=0.2,
                        temperature=None,
                        sliding_window_size=None, # required for
                                                  # "sliding_window_average"
                        )

Then, you can solve the optimization problem by running `solve()` method. If you want to follow the result step by step, you can set `solve(disp=True)`:

In [5]:
results = optimizer.solve(disp=True)

Generation     0: f(x) = 482.03001167132146065342, objective function has been called 0 times
Generation     1: f(x) = 482.03001167132146065342, objective function has been called 24 times
Generation     2: f(x) = 482.03001167132146065342, objective function has been called 48 times
Generation     3: f(x) = 482.03001167132146065342, objective function has been called 72 times
Generation     4: f(x) = 427.48803936091911737094, objective function has been called 96 times
Generation     5: f(x) = 356.35203356042273981075, objective function has been called 120 times
Generation     6: f(x) = 235.94948185303601917440, objective function has been called 144 times
Generation     7: f(x) = 235.94948185303601917440, objective function has been called 168 times
Generation     8: f(x) = 235.94948185303601917440, objective function has been called 192 times
Generation     9: f(x) = 235.94948185303601917440, objective function has been called 216 times
Generation    10: f(x) = 180.70581159526716419

## Step 4 (Optional)

If you want to save the results permanently, you can use `save()` method. It will save the configurations, the stop conditions, the iteration history, and the final optimum to a JSON file. (In this example, a file named "BBOB2015-F7(I1), D=10_OACDE" will be created in the current folder.)

In [6]:
optimizer.save()

## Step 5 (Optional)

Or you can just view the results on your screen like:

In [7]:
tok = time.time()
print("\tElapsed Time: {:.2f}s".format(tok-tik))
print("\tCalculated results:", results)
print("\tTheoretical optimal value:", problem.f_opt_theory)
print("\tRelative error:", results[1]-problem.f_opt_theory)

print(optimizer.history) # show the iteration history

	Elapsed Time: 3.58s
	Calculated results: (array([-0.04165458,  0.76147019,  0.27006614,  3.18187312,  2.44765681,
        3.76168523,  1.70426389, -2.38315949,  3.66923105,  2.05417579]), 92.97516698441356)
	Theoretical optimal value: 92.94
	Relative error: 0.03516698441356425
       nth    FES  ...                                            used_CR     f_error
0        0      0  ...                                                NaN  389.090012
1        1     24  ...  [0.4012595008036087, 0.09961493022127133, 0.86...  389.090012
2        2     48  ...  [0.6464559113657224, 0.16121821329251806, 0.19...  389.090012
3        3     72  ...  [0.4286859808129494, 0.6009477895248134, 0.157...  389.090012
4        4     96  ...  [0.15966305092308175, 0.9418834886956645, 0.30...  334.548039
...    ...    ...  ...                                                ...         ...
996    996  23904  ...  [0.0021730418971608323, 0.9440408288781463, 0....    0.035167
997    997  23928  ...  [0.561274

## References

[1] Nikolaus Hansen, Anne Auger, Raymond Ros, Olaf Mersmann, Tea Tušar & Dimo Brockhoff (2021) COCO: a platform for comparing continuous optimizers in a black-box setting, Optimization Methods and Software, 36:1, 114-144, DOI: [10.1080/10556788.2020.1808977](https://doi.org/10.1080/10556788.2020.1808977)